<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
 

<strong>Objectives:</strong>
+ Descartar Productos descatalogados
    
    
<strong>Conclusions:</strong>
+ Descartamos los productos bajo las siguientes circunstancias:
    + Ninguna venta posterior al 01/03/2020
    + Sin datos de stock para ninguna fecha
   
</div>

In [1]:
import os, sys

def get_root_path(search_word):
    """ Búsqueda hacia atrás de un directorio con nombre search_word. """
    cwd = os.getcwd()
    while str(os.getcwd()).split('/')[-1] != search_word:
        os.chdir("..")
        cwd = os.getcwd()
    return cwd

ROOT_DIR = get_root_path('TFM_StockForecast')

sys.path.insert(0, ROOT_DIR) # Poner en el primer lugar del PATH (evitar paquetes con mismos nombres en otros directorios)

In [5]:
import pandas as pd
import numpy as np
from src.load_data import load_raw_csv

In [6]:
df = load_raw_csv("ventas")

df

Reading CSV in data/raw/01_TablaVentas.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 0.


,fecha,producto,udsventa
49871,2015-06-29,24,1151.28
3463,2015-06-29,18,771.21
82457,2015-06-29,97,201.72
51787,2015-06-29,57,1119.30
47460,2015-06-29,22,423.12
...,...,...,...
67629,2020-03-21,80,693.72
1826,2020-03-21,14,6239.79
49824,2020-03-21,24,2555.94
20971,2020-03-21,30,246.00


## Productos descatalogados

In [8]:
ventas_byprod = pd.read_csv("data/clean/ventas_byprod.csv")
stock_byprod = pd.read_csv("data/clean/stock_byprod.csv")

ventas_byprod = ventas_byprod.loc[:,["producto","fecha_primera_venta","fecha_ultima_venta"]]
stock_byprod = stock_byprod.loc[:,["producto","fecha_primer_stock","fecha_ultimo_stock"]]
prod = ventas_byprod.merge(stock_byprod, how ='outer', on='producto')

# PRODUCTOS SIN DATOS DE STOCK
prod.loc[prod.fecha_primer_stock.isna()]

,producto,fecha_primera_venta,fecha_ultima_venta,fecha_primer_stock,fecha_ultimo_stock
24,33,2015-06-29,2020-03-20,NaN,NaN
25,34,2015-06-30,2020-03-20,NaN,NaN
70,81,2015-07-03,2020-03-20,NaN,NaN
72,83,2016-08-22,2020-03-20,NaN,NaN
80,92,2016-04-22,2020-03-18,NaN,NaN
81,93,2016-01-19,2020-01-13,NaN,NaN


In [11]:
# PRODUCTOS SIN VENTAS DESDE 01-03-2020
prod.dropna().loc[prod.fecha_ultima_venta < '2020-03-01']

,producto,fecha_primera_venta,fecha_ultima_venta,fecha_primer_stock,fecha_ultimo_stock
7,16,2015-06-29,2020-01-23,2019-06-05,2020-01-23
34,44,2015-06-29,2019-11-05,2019-06-05,2019-10-05
37,47,2017-01-24,2020-01-23,2019-06-05,2020-01-23
44,54,2015-06-29,2019-12-23,2019-06-05,2019-10-03
58,7,2015-06-29,2019-06-21,2019-06-05,2019-07-30
59,70,2015-06-29,2020-02-24,2019-06-05,2019-06-05
60,71,2015-06-29,2019-12-24,2019-06-05,2020-02-27
64,75,2015-06-30,2020-02-21,2019-06-05,2020-02-26
65,76,2016-11-02,2019-12-26,2019-06-11,2019-10-21
71,82,2015-06-29,2019-11-29,2019-06-05,2019-12-21
